**Install SQL**

In [1]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.1 MB/s eta 0:00:00


**Load Sql Extensiont**

In [2]:
%load_ext sql

**Create SQLite database**

In [3]:
%sql sqlite://

In [5]:
%%sql


CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');


CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');


CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');


 * sqlite://
Done.
15 rows affected.
Done.
3 rows affected.
Done.
2 rows affected.


[]

In [6]:
#1) What is the total amount each customer spent at the restaurant?
%%sql
with total_amt as(
  select sales.customer_id,
  sales.product_id,
  menu.price from
  sales join
  menu on
  sales.product_id = menu.product_id
)
select
customer_id,
sum(price) as amount
from total_amt
group by customer_id
order by customer_id;

 * sqlite://
Done.


customer_id,amount
A,76
B,74
C,36


In [7]:
#2) How many days has each customer visited the restaurant?
%%sql
with no_of_days as(
  select customer_id,
  sales.order_date as visit_date
  from sales
 )
 select customer_id,
 count(distinct(visit_date)) as total_days
 from no_of_days
 group by customer_id;

 * sqlite://
Done.


customer_id,total_days
A,4
B,6
C,2


In [8]:
#3. What was the first item from the menu purchased by each customer?
%%sql
with first_item as(
  select sales.customer_id,
  sales.order_date,
  row_number() over(partition by sales.customer_id order by sales.order_date, sales.product_id) as rn,
  menu.product_name
  from sales
  join menu on
  sales.product_id=menu.product_id
)
select customer_id,
product_name from first_item
where rn =1;

 * sqlite://
Done.


customer_id,product_name
A,sushi
B,curry
C,ramen


In [9]:
#4.  What is the most purchased item on the menu and how many times was it purchased by all customers?
%%sql
WITH items_cte AS (
  SELECT
    sales.product_id,
    menu.product_name,
    COUNT(*) AS purchase_count
  FROM
    sales
  JOIN
   menu
    ON sales.product_id = menu.product_id
  GROUP BY
    sales.product_id,
    menu.product_name
)
SELECT
  product_name,
  purchase_count
FROM
  items_cte
ORDER BY
  purchase_count DESC
LIMIT 1;

 * sqlite://
Done.


product_name,purchase_count
ramen,8


In [11]:
#5. Which item was the most popular for each customer?
%%sql
with cte as(
  select sales.customer_id,
  sales.product_id,
  menu.product_name,
  count(*) as prod_count,
  row_number() over(partition by customer_id order by count(*) desc) as r
  from sales
  join menu
  on sales.product_id = menu.product_id
  group by sales.customer_id,sales.product_id,menu.product_name
)
select
customer_id,
product_name
from cte where r=1;

 * sqlite://
Done.


customer_id,product_name
A,ramen
B,sushi
C,ramen


In [12]:
#6. Which item was purchased first by the customer after they became a member?
%%sql
WITH membership_cte AS (
  SELECT
    sales.customer_id,
    sales.product_id,
    sales.order_date,
    members.join_date
  FROM
    sales
  JOIN
    members
  ON
    sales.customer_id = members.customer_id
)
, ranked_purchases AS (
  SELECT
    customer_id,
    product_id,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) AS rn
  FROM
    membership_cte
  WHERE
    order_date >= join_date
)
SELECT
  customer_id,
  product_id
FROM
  ranked_purchases
WHERE
  rn = 1;


 * sqlite://
Done.


customer_id,product_id
A,2
B,1


In [13]:
#7. Which item was purchased just before the customer became a member?
%%sql
WITH membership_cte AS (
  SELECT
    sales.customer_id,
    sales.product_id,
    sales.order_date,
    members.join_date
  FROM
    sales
  JOIN
    members
  ON
    sales.customer_id = members.customer_id
),
ranked_purchases AS (
  SELECT
    customer_id,
    product_id,
    order_date,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date DESC) AS rn
  FROM
    membership_cte
  WHERE
    order_date < join_date
)
SELECT
  customer_id,
  product_id
FROM
  ranked_purchases
WHERE
  rn = 1;

 * sqlite://
Done.


customer_id,product_id
A,1
B,1


In [14]:
# 8. What is the total items and amount spent for each member before they became a member?
%%sql
WITH prd_amt_cte AS (
  SELECT
    sales.customer_id,
    sales.product_id,
    sales.order_date,
    menu.price,
    members.join_date
  FROM
    sales
  JOIN
    menu
  ON
    sales.product_id = menu.product_id
  LEFT JOIN
    members
  ON
    sales.customer_id = members.customer_id
),
bfr_mem_cte AS (
  SELECT
    customer_id,
    product_id,
    price
  FROM
    prd_amt_cte
  WHERE
    order_date < join_date
)
SELECT
  customer_id,
  COUNT(product_id) as total_items,
  SUM(price) as amnt_spent
FROM
  bfr_mem_cte
GROUP BY
  customer_id;


 * sqlite://
Done.


customer_id,total_items,amnt_spent
A,2,25
B,3,40


In [15]:
#9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
%%sql
WITH Combined_cte AS (
  SELECT
    sales.customer_id,
    sales.product_id,
    menu.price,
    menu.product_name
  FROM
    sales
  JOIN
    menu
  ON
    sales.product_id = menu.product_id
),
points_cte AS (
  SELECT
    customer_id,
    CASE
      WHEN product_name = 'sushi' THEN 2 * price * 10
      ELSE price * 10
    END AS points
  FROM
    Combined_cte
)
SELECT
  customer_id,
  SUM(points) AS total_points
FROM
  points_cte
GROUP BY
  customer_id;

 * sqlite://
Done.


customer_id,total_points
A,860
B,940
C,360


In [18]:
#10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?
%%sql
WITH joining_mem AS (
  SELECT
    sales.customer_id,
    sales.product_id,
    sales.order_date,
    menu.price,
  	menu.product_name,
    members.join_date
  FROM
    sales
  JOIN
    menu
  ON
    sales.product_id = menu.product_id
  JOIN
    members
  ON
    sales.customer_id = members.customer_id
),
points_cte AS (
  SELECT
    customer_id,
    CASE
      WHEN order_date BETWEEN join_date AND date(join_date, '+6 day') THEN 2 * price * 10
      WHEN product_name = 'sushi' THEN 2 * price * 10
      ELSE price * 10
    END AS points
  FROM
    joining_mem
  WHERE
    order_date <= '2023-01-31'
)
SELECT
  customer_id,
  SUM(points) AS total_points
FROM
  points_cte
WHERE
  customer_id IN ('A', 'B')
GROUP BY
  customer_id;

 * sqlite://
Done.


customer_id,total_points
A,1370
B,940
